# Assignment - Basic Pandas
<sup>Created by Natawut Nupairoj, Department of Computer Engineering, Chulalongkorn University</sup>

Using pandas to explore youtube trending data from GB (GBvideos.csv and GB_category_id.json) and answer the questions.

In [ ]:
import pandas as pd
import numpy as np

To simplify data retrieval process on Colab, we heck if we are in the Colab environment and download data files from a shared drive and save them in folder "data".

For those using jupyter notebook on the local computer, you can read data directly assuming you save data in the folder "data".

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !wget https://www.dropbox.com/s/7u5j5uei57yyksh/data.tgz?dl=0 -O data.tgz
    !tar -xzvf data.tgz

--2021-01-29 14:19:11--  https://www.dropbox.com/s/7u5j5uei57yyksh/data.tgz?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7u5j5uei57yyksh/data.tgz [following]
--2021-01-29 14:19:11--  https://www.dropbox.com/s/raw/7u5j5uei57yyksh/data.tgz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc51a46885950af63e09a3766bc0.dl.dropboxusercontent.com/cd/0/inline/BH6PX1D3GpRTDDnKussDN9nthpRffCnhlVs-o3AL2u0N0HUwOXPzB7LjNnbCOe9wWOUni202sHnTvhakkl56IFxWZlIyGHMp9a_2MtjlZ-ruSMGi6Oy42LxQFaHuETItK7o/file# [following]
--2021-01-29 14:19:12--  https://uc51a46885950af63e09a3766bc0.dl.dropboxusercontent.com/cd/0/inline/BH6PX1D3GpRTDDnKussDN9nthpRffCnhlVs-o3AL2u0N0HUwOXPzB7LjNnbCOe9wWOUni202sHnTvhakkl56IFxWZlIyGHMp9a_2MtjlZ-ruSMGi6Oy42L

## How many rows are there in the GBvideos.csv after removing duplications?

In [ ]:
gb_vdo_df = pd.read_csv('data/GBvideos.csv')
gb_vdo_df.drop_duplicates(inplace=True)
gb_vdo_df.shape

(38745, 16)

So, There are **38745** rows after removing duplications.

## How many VDO that have "dislikes" more than "likes"?  Make sure that you count only unique title!

In [ ]:
more_dislikes_vdo = gb_vdo_df[(gb_vdo_df.dislikes>gb_vdo_df.likes)]
unique_title = more_dislikes_vdo.title.unique()
len(unique_title)

56

So, There are **56** videos that have "dislike" more than "likes"

## How many VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments?

In [ ]:
gb_vdo_df['trending_dt'] = pd.to_datetime(gb_vdo_df.trending_date, format='%y.%d.%m', errors='ignore', utc=True)
comment_more = gb_vdo_df[gb_vdo_df.comment_count>10000]
gb_vdo_df_groupby_trending = comment_more.groupby('trending_dt') 
trending_22_jan_2018 = gb_vdo_df_groupby_trending.get_group('2018-01-22 00:00:00+00:00')
len(trending_22_jan_2018)

28

So, There are **28** videos that are trending on 22 Jan 2018 with comments more than 10,000 comments.

## Which date that has the minimum average number of comments per VDO?

In [ ]:
avg_comment_each_date=gb_vdo_df_groupby_trending.mean()['comment_count']
sum = gb_vdo_df_groupby_trending.sum()['comment_count']
min_avg_comment_each_date = min(avg_comment_each_date)
for date, group in gb_vdo_df_groupby_trending:
  avg_group = group['comment_count'].mean()
  if (min_avg_comment_each_date == avg_group):
    print("date: ",date)
    print("group average: ",avg_group)

date:  2017-11-14 00:00:00+00:00
group average:  26409.115384615383


So, **14/11/2017** is the date that has minimum average number of comments per VDO which is around **26,409** comments per VDO

## Compare "Sports" and "Comady", how many days that there are more total daily views of VDO in "Sports" category than in "Comady" category?

In [ ]:
import json
with open('data/GB_category_id.json') as fd:
  cat = json.load(fd)

In [ ]:
cat_list = []
for d in cat['items']:
  cat_list.append((int(d['id']), d['snippet']['title']))

In [ ]:
cat_df = pd.DataFrame(cat_list, columns=['id', 'category'])
gb_vdo_df_withcat = gb_vdo_df.merge(cat_df, left_on='category_id', right_on='id')

In [ ]:
gb_vdo_df_withcat_grouby_date = gb_vdo_df_withcat.groupby(["trending_dt","category"])
date_sports_more_comedy = []
sport_views = 0
comedy_views =0
date = ""
for col, group in gb_vdo_df_withcat_grouby_date:
  if col[0] == date and sport_views>comedy_views:
      date_sports_more_comedy.append(date)
  else:
    if  col[1] == "Sports":
      sport_views = float(group.views.sum())
      date =col[0]
    elif col[1] == "Comedy":
      comedy_views =  float(group.views.sum())
      date = col[0]

len(date_sports_more_comedy)

780

So, There are **780** days that Sports views is higher than Comedy views.